In [100]:
#Reference method and code from here:
#https://fairlearn.org/v0.7.0/quickstart.html


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

data = fetch_openml(data_id=1590, as_frame=True)
X = pd.get_dummies(data.data)
y_true = (data.target == '>50K') * 1
sex = data.data['sex']
sex.value_counts()

Male      32650
Female    16192
Name: sex, dtype: int64

In [101]:
from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(min_samples_leaf=10, max_depth=4)
classifier.fit(X, y_true)

y_pred = classifier.predict(X)

gm = MetricFrame(metrics = accuracy_score, y_true = y_true, y_pred= y_pred, sensitive_features=sex)
print(gm.overall)

0.8443552680070431


In [102]:
print(gm.by_group)

sex
Female    0.925148
Male      0.804288
Name: accuracy_score, dtype: object


In [103]:
#Examine selection rate across sexes
from fairlearn.metrics import selection_rate

sr = MetricFrame(metrics = selection_rate, y_true = y_true, y_pred= y_pred, sensitive_features=sex)

print(sr.overall)
print(sr.by_group)

0.16385487899758405
sex
Female     0.06355
Male      0.213599
Name: selection_rate, dtype: object


In [104]:
#Create new model to mitigate for disparate selection rate by modifying Demographic Parity metric

from fairlearn.reductions import ExponentiatedGradient, DemographicParity

np.random.seed(0)

constraint = DemographicParity()
classifier = DecisionTreeClassifier(min_samples_leaf=10, max_depth=4)
mitigator = ExponentiatedGradient(classifier, constraint)

mitigator.fit(X, y_true, sensitive_features=sex)

y_pred_mitigated = mitigator.predict(X)

sr_mitigated = MetricFrame(metrics = selection_rate, y_true=y_true, y_pred=y_pred_mitigated, sensitive_features=sex)

print(sr_mitigated.overall)
print(sr_mitigated.by_group)


0.16614798738790384
sex
Female    0.155262
Male      0.171547
Name: selection_rate, dtype: object


In [105]:
#%pip install aif360

import numpy as np
np.random.seed(0)

from aif360.datasets import AdultDataset
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.metrics import BinaryLabelDatasetMetric

In [106]:
#Reference method and code from here:
#https://github.com/Trusted-AI/AIF360/blob/master/examples/demo_adversarial_debiasing.ipynb

#Compute fairness metric on original training data
data_orig = load_preproc_data_adult()

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

data_orig_train, data_orig_test = data_orig.split([0.7], shuffle=True)

metric_orig_train = BinaryLabelDatasetMetric(data_orig_train, 
                                      unprivileged_groups=unprivileged_groups,
                                      privileged_groups=privileged_groups)

metric_orig_test = BinaryLabelDatasetMetric(data_orig_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

#Mean differences between 'male' and 'female' groups
print(metric_orig_train.mean_difference())
print(metric_orig_test.mean_difference())

-0.1930752110970722
-0.19804754300152455


In [107]:
#Apply scaling to dataset
from sklearn.preprocessing import StandardScaler, MaxAbsScaler

min_max_scaler = MaxAbsScaler()
data_orig_train.features = min_max_scaler.fit_transform(data_orig_train.features)
data_orig_test.features = min_max_scaler.transform(data_orig_test.features)

#Scale train
metric_scaled_train = BinaryLabelDatasetMetric(data_orig_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

#Scale test
metric_scaled_test = BinaryLabelDatasetMetric(data_orig_test, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

print(metric_scaled_train.mean_difference())
print(metric_scaled_test.mean_difference())


-0.1930752110970722
-0.19804754300152455


In [108]:
#Learn adversarial debiasing classifier without debiasing 
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
import tensorflow.compat.v1 as tf
tf.reset_default_graph() 
tf.disable_eager_execution()

biased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False, 
                           sess=tf.Session())

biased_model.fit(data_orig_train)


epoch 0; iter: 0; batch classifier loss: 0.692007
epoch 0; iter: 200; batch classifier loss: 0.402107
epoch 1; iter: 0; batch classifier loss: 0.437729
epoch 1; iter: 200; batch classifier loss: 0.504335
epoch 2; iter: 0; batch classifier loss: 0.394550
epoch 2; iter: 200; batch classifier loss: 0.488978
epoch 3; iter: 0; batch classifier loss: 0.467837
epoch 3; iter: 200; batch classifier loss: 0.393646
epoch 4; iter: 0; batch classifier loss: 0.406567
epoch 4; iter: 200; batch classifier loss: 0.381459
epoch 5; iter: 0; batch classifier loss: 0.385749
epoch 5; iter: 200; batch classifier loss: 0.428927
epoch 6; iter: 0; batch classifier loss: 0.455799
epoch 6; iter: 200; batch classifier loss: 0.411776
epoch 7; iter: 0; batch classifier loss: 0.314866
epoch 7; iter: 200; batch classifier loss: 0.436166
epoch 8; iter: 0; batch classifier loss: 0.430930
epoch 8; iter: 200; batch classifier loss: 0.401677
epoch 9; iter: 0; batch classifier loss: 0.488311
epoch 9; iter: 200; batch classi

In [109]:
from aif360.metrics import ClassificationMetric

#Apply biased model to test data
data_nodebiasing_test = biased_model.predict(data_orig_test)

#Report mean differences between 'male' and 'female' groups
metric_nodebiasing_test = BinaryLabelDatasetMetric(data_nodebiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)


print(metric_nodebiasing_test.mean_difference())

#Report classification accuracy
classified_metric_nodebiasing_test = ClassificationMetric(data_orig_test, 
                                                 data_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)

print(classified_metric_nodebiasing_test.accuracy())

#Report disparate impact ratio
print(classified_metric_nodebiasing_test.disparate_impact())


-0.20557244174265452
0.8042039172865625
0.0


In [110]:
#Conduct fairness metric analysis after fitting model with debiasing option
tf.Session().close()
tf.reset_default_graph()


debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True, 
                           sess=tf.Session())

debiased_model.fit(data_orig_train)


epoch 0; iter: 0; batch classifier loss: 0.685663; batch adversarial loss: 0.866392
epoch 0; iter: 200; batch classifier loss: 0.829415; batch adversarial loss: 0.957977
epoch 1; iter: 0; batch classifier loss: 0.948465; batch adversarial loss: 0.914112
epoch 1; iter: 200; batch classifier loss: 0.661319; batch adversarial loss: 0.757911
epoch 2; iter: 0; batch classifier loss: 0.510904; batch adversarial loss: 0.698530
epoch 2; iter: 200; batch classifier loss: 0.383084; batch adversarial loss: 0.654228
epoch 3; iter: 0; batch classifier loss: 0.353087; batch adversarial loss: 0.658804
epoch 3; iter: 200; batch classifier loss: 0.416485; batch adversarial loss: 0.649679
epoch 4; iter: 0; batch classifier loss: 0.444525; batch adversarial loss: 0.614053
epoch 4; iter: 200; batch classifier loss: 0.472028; batch adversarial loss: 0.676581
epoch 5; iter: 0; batch classifier loss: 0.500998; batch adversarial loss: 0.633211
epoch 5; iter: 200; batch classifier loss: 0.458376; batch adversa

In [111]:
#Apply debiased model to test data
data_debiased_test = debiased_model.predict(data_orig_test)

#Report debiased mean differences between 'male' and 'female' groups
metric_debiased_test = BinaryLabelDatasetMetric(data_debiased_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print(metric_debiased_test.mean_difference())

#Report debiased classification accuracy
classified_metric_debiased_test = ClassificationMetric(data_orig_test, 
                                                 data_debiased_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)

print(classified_metric_debiased_test.accuracy())

#Report disparate impact ratio
print(classified_metric_debiased_test.disparate_impact())

-0.08326598276020208
0.7913737801132874
0.588258892864131
